In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "Hi, what would be the best copy for a {business} website?"
)
prompt = template.format(business="golf lounge")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

"Welcome to the Ultimate Golf Lounge Experience!\n\nAt [Golf Lounge Name], we are passionate about creating a haven for golf enthusiasts like you. Step into our world-class facility and indulge in the perfect blend of golf, relaxation, and entertainment. Whether you're a seasoned golfer or just starting out, our golf lounge is designed to cater to all skill levels and provide an unforgettable experience.\n\nUnleash your swing on our state-of-the-art simulators, which offer a realistic and immersive golfing experience. Play on world-renowned courses, challenge your friends to friendly competitions, or even receive professional coaching from our expert instructors. Our cutting-edge technology ensures that every swing is accurately measured, allowing you to fine-tune your skills and track your progress.\n\nBut it's not just about the game. Our luxurious lounge area provides a comfortable and stylish setting for you to unwind and socialize. Relax in plush seating, enjoy a refreshing drink 

In [2]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

template = ChatPromptTemplate.from_messages([
    ("system", "You are a copywriter for digital products. You only reply in {language}."),
    ("ai", "Hi, I am your chatbot."),
    ("human", "Hi, what would be the best copy for a {business} website?")])


prompt = template.format_messages(
    language="Spanish",
    business="Coffe shop"
)

chat.predict_messages(prompt)

AIMessage(content='¡Bienvenidos al mejor lugar para disfrutar del café! En nuestra cafetería, encontrarás el equilibrio perfecto entre sabor, calidad y ambiente acogedor. Nuestros expertos baristas están listos para deleitarte con una amplia variedad de cafés de origen, cuidadosamente seleccionados y preparados con pasión. Además, ofrecemos deliciosos postres y bocadillos que complementarán tu experiencia. Ven y descubre por qué somos el destino favorito de los amantes del café. ¡Te esperamos con una taza de café perfecta!')

In [5]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(',')
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, How, Are,You,?")

['Hello', 'How', 'Are', 'You', '?']

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. You should answer only in list of maximum {max}, separated with a comma. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max=10,
    question="What are the names of planets?"
)

message = chat.predict_messages(prompt).content

p = CommaOutputParser()
planets = p.parse(message)
print(planets)

# planets

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']


In [19]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max": 20,
    "question": "What are the names of Pokemons?"
})

['Pikachu',
 'Bulbasaur',
 'Charizard',
 'Squirtle',
 'Jigglypuff',
 'Eevee',
 'Snorlax',
 'Mewtwo',
 'Gyarados',
 'Dragonite',
 'Blastoise',
 'Venusaur',
 'Lapras',
 'Arcanine',
 'Gengar',
 'Machamp',
 'Alakazam',
 'Mew',
 'Zapdos',
 'Articuno']

In [ ]:
chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a chef. You create recipes that are easy to follow for any type of cuisine with ingredients easy to find."),
    ("human", "I want to cook {cuisine} food. Give me a simple recipe and ingredients.")
])